# Imports 

In [1]:
%load_ext autoreload
%autoreload 2

In [59]:
import sys, os
insertIntoPath(os.path.join(os.getcwd(), '..', '..', 'playground'))
insertIntoPath(os.path.join(os.getcwd(), '..', 'price'))

In [60]:
def insertIntoPath(newPath):
    if newPath not in sys.path:
        sys.path.insert(0, newPath)

In [61]:
import tweepy
import numpy as np
import pandas as pd
import time
from datetime import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from datetime import datetime
import re
from dateutil import parser

In [63]:
import twitterUtils

In [65]:
api = twitterUtils.getTwitterApi()
twitterUtils.filterTwitterData(twitterUtils.getCurrentTwitterData(api), "(F|f)acebook")

/home/brandon/Desktop/StockBot/data/twitter/twitterUtils.py:94: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  company_tweets = data.loc[data['Texts'].str.contains(company_regex, na=False)]


,Datetime,IDs,Sentiment,Sources
448,2018-07-06 21:59:20+00:00,1015354537723428870,-0.4404,CNBCtech
400,2018-07-06 12:04:00+00:00,1015204716832350209,-0.4019,ForbesTech
251,2018-07-06 08:35:58+00:00,1015152361176694784,0.0000,BBCWorldBiz
488,2018-07-04 18:50:03+00:00,1014582127982272512,-0.2732,CNBCtech
512,2018-07-03 09:48:23+00:00,1014083422577360896,0.4215,CNBCtech
62,2018-07-03 07:00:37+00:00,1014041201622814720,-0.3400,technology
515,2018-07-02 22:08:53+00:00,1013907386166382592,0.0000,CNBCtech
287,2018-07-02 19:24:19+00:00,1013865975652012034,-0.2263,BBCWorldBiz
524,2018-07-02 18:29:04+00:00,1013852069776777216,-0.3612,CNBCtech
532,2018-07-02 13:41:26+00:00,1013779684147056643,-0.1027,CNBCtech


# Twitter Login

In [6]:
consumer_token = "FSdktl3u8Q4xhmEHeC7tt4Q6P"
consumer_secret = "pErt27djL4HCcBt0z9PU6w7zVFygfVsEP1Aqx2gQ70uYibqP7p"
access_key = "1902120404-I1QEwvP7uLEBRJefMMPdPSf4uChDnsNvzO8hFcj"
access_secret = "RZAp4AlaXoszBXqp8gkMJM1ed6TJzTBFCDS99248hV9i4"

In [7]:
auth = tweepy.OAuthHandler(consumer_token, consumer_secret)
auth.set_access_token(access_key, access_secret)

In [8]:
api = tweepy.API(auth)

# Main

In [12]:
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except (tweepy.RateLimitError, tweepy.TweepError):
            print("Rate limit exceeded, I'm passing out for a bit peace")
            time.sleep(15 * 60)

In [15]:
def makeCleanRep(origSource, removeLink=True):
    """
    Returns a list of the sid, text, and date of a given source.
    """
    
    sid = origSource._json['id']
    text = origSource._json['full_text']
    date = origSource._json['created_at']
    
    if removeLink==True:
        nonLinks = []
        for word in text.split(" "):
            if  "http" not in word:
                nonLinks.append(word)
        text = " ".join(nonLinks)
            
    
    return [sid, text, date]

In [54]:
def getCurrentTwitterData():
    allData = []
    ids = []
    texts = []
    dates = []
    sources = []
    handles = ["technology", "business", "WSJbusiness", "BBCWorldBiz", "ForbesTech", "CNBCtech"]
    c = 0

    for handle in handles:
#         print("Starting", handle,  "...")
        hc = 0
        cur = tweepy.Cursor(api.user_timeline, screen_name = handle, tweet_mode='extended').items(100)
        for source in limit_handled(cur):
            clean = makeCleanRep(source)
            if clean[1] not in texts:
                ids.append(clean[0])
                texts.append(clean[1])
                dates.append(clean[2])
                sources.append(handle)
                c += 1
                hc += 1
#         print("Gathered", hc, "tweets for", handle)

    sia = SIA()
    df = pd.DataFrame({"IDs":ids, "Texts":texts, "Dates":dates, "Sources": sources}, columns=["IDs", "Texts", "Dates", "Sources"])
    df['Sentiment'] = pd.Series([sia.polarity_scores(text)['compound'] for text in df['Texts']])
    return df

In [51]:
def filterTwitterData(data, company_regex):
    """
    :param csv: csv file name
    :param company_regex: regex to select company
    :return: dataframe with only dates and sentiment of tweets sorted by date
    """
    company_tweets = data.loc[data['Texts'].str.contains(company_regex, na=False)]
    company_tweets = company_tweets[['Dates', 'Sentiment', 'IDs', 'Sources']]
    to_datetime_data = [parser.parse(date) for date in company_tweets['Dates']]
    company_tweets['Datetime'] = to_datetime_data
    company_tweets = company_tweets.sort_values(by = 'Datetime', ascending=False)
    return company_tweets[['Datetime', 'IDs', 'Sentiment', 'Sources']]

In [56]:
filterTwitterData(getCurrentTwitterData(), "(F|f)acebook")

/home/brandon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: generator 'limit_handled' raised StopIteration
  
/home/brandon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys


,Datetime,IDs,Sentiment,Sources
448,2018-07-06 21:59:20+00:00,1015354537723428870,-0.4404,CNBCtech
400,2018-07-06 12:04:00+00:00,1015204716832350209,-0.4019,ForbesTech
251,2018-07-06 08:35:58+00:00,1015152361176694784,0.0000,BBCWorldBiz
488,2018-07-04 18:50:03+00:00,1014582127982272512,-0.2732,CNBCtech
512,2018-07-03 09:48:23+00:00,1014083422577360896,0.4215,CNBCtech
62,2018-07-03 07:00:37+00:00,1014041201622814720,-0.3400,technology
515,2018-07-02 22:08:53+00:00,1013907386166382592,0.0000,CNBCtech
287,2018-07-02 19:24:19+00:00,1013865975652012034,-0.2263,BBCWorldBiz
524,2018-07-02 18:29:04+00:00,1013852069776777216,-0.3612,CNBCtech
532,2018-07-02 13:41:26+00:00,1013779684147056643,-0.1027,CNBCtech


In [ ]:
def createCompanyRefDict(money, regex):
    return {'money': money, 'shares':0, 'regex':regex, 'buy_state':True, 'last_sell':0, 'last_buy':0}

In [ ]:
SELL_PRED = 0
BUY_PRED = 1


refData = {"FB": makeRefDataDict(money, shares, regex, buy_state, last_sell, last_buy)}

while True:
    api = twitterUtils.getTwitterApi()
    twitter_data = twitterUtils.getCurrentTwitterData(api)
    for symbol in refData.keys():
        filtered_twitter_data = filterTwitterData(COMPANY_REGEX)
        price_data = getCurrentPriceData()
        stockPrice = #ASDFASDFASDFASDF
        inputData = formatInput(price_data, inputData)
        pred = bot.neural_net.predict(data)

        if pred == BUY_PRED and refData[symbol]['buy_state'] == True:
            sharesToBuy = int(refData[symbol]['money'] / stockPrice)
            stockPrice = buyStock(symbol, sharesToBuy)
            refData[symbol]['shares'] += sharesToBuy
            refDatta[symbol]['money'] -= sharesToBuy * stockPrice
            
        elif pred == SELL_PRED and refData[symbol]['buy_state'] == False:
            sharesToSell = refData[symbol]['shares']
            stockPrice = sellStock(symbol, sharesToSell)
            refData[symbol]['shares'] -= sharesToBuy
            refDatta[symbol]['money'] += sharesToBuy * stockPrice
    
    
    sleep(10)